In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import os

In [2]:
print(os.listdir("../data/raw"))

['.gitkeep', 'user_logs_v2.csv']


In [3]:
def reducedatatype(df):
    int_col = list(df.select_dtypes(include=[int]).columns)
    for col in int_col:
        if((np.min(df[col]) >= -127) and (np.max(df[col]) <= 127)):
            df[col] = df[col].astype(np.int8)
        elif((np.min(df[col]) >= -32768) and (np.max(df[col]) <= 32768)):
            df[col] = df[col].astype(np.int16)
        elif((np.min(df[col]) >= -2147483647) and (np.max(df[col]) <= 2147483647)):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)
    int_float = list(df.select_dtypes(include=[float]).columns)
    for col in int_float:
        df[col] = df[col].astype(np.float32)

In [2]:
user_log_mar = pd.read_csv('../data/raw/user_logs_v2.csv',parse_dates=['date'])

In [3]:
user_log_mar.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,2017-03-31,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,2017-03-30,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,2017-03-31,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,2017-03-31,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,2017-03-31,2,1,0,1,112,93,28401.558


In [7]:
q1 = user_log_mar['num_25'].quantile(0.25)
q2 = user_log_mar['num_25'].quantile(0.75)
IQR = q2-q1
print(q1-1.5*IQR,q2+1.5*IQR)

-10.5 17.5


In [19]:
user_log_mar[user_log_mar['num_25']> 17.5]

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,2017-03-31,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,2017-03-31,176,4,2,2,19,191,7100.454
7,JEjl2W1ivEI6epeobW3n5rDg/r6PJm4OJhtmhwYaQIc=,2017-03-31,181,68,5,3,54,291,22433.105
11,6eT/NGMljpNgEeqTYIdhkc/dvUik+b+41sBNrXJ0cBQ=,2017-03-24,78,5,3,2,8,85,3490.020
15,NCRcE4eNurBdOoRk0m+hwDz94xBN6+KL5PTwen4hmWU=,2017-03-25,40,6,2,2,131,149,29400.889
...,...,...,...,...,...,...,...,...,...
18394396,YEShno0C2cu5pp7lxkRh8Bdwvi7+yLIPKXOuh9f7WOU=,2017-03-29,18,2,0,2,25,43,6618.616
18394450,mvlETFqbWa0++0FCiLOYCKFmnrmF/vRfJT1Us8ugYgs=,2017-03-06,28,1,0,0,19,25,4518.310
18394514,yclCWrg8je62sq3SZKCr3Wyn9Rh1py4T7lDgQ/Uwy6c=,2017-03-11,19,2,0,0,7,28,1923.509
18394624,xj8QAscgaOggWMJNZ0D0h2h/IaYCop40G9fVhTNeV0Q=,2017-03-05,19,2,0,2,25,35,7339.135


In [ ]:
chunksize = 10**6
first_chunk = True
for chunk in pd.read_csv('../data/raw/user_logs.csv',chunksize=chunksize,parse_dates=['date']):
    chunk_feb = chunk[(chunk['date']>='2017-02-01') & (chunk['date']<='2017-02-28')]
    chunk_jan = chunk[(chunk['date']>='2017-01-01') & (chunk['date']<='2017-01-31')]
    if(first_chunk == True):
        chunk_feb.to_csv('../data/processed/feb_log.csv', mode='a',index=False)
        chunk_jan.to_csv('../data/processed/jan_log.csv', mode='a',index=False)
        first_chunk = False
    else:
        chunk_feb.to_csv('../data/processed/feb_log.csv', mode='a', header=False,index=False)
        chunk_jan.to_csv('../data/processed/jan_log.csv', mode='a', header=False,index=False)

In [6]:
user_log_jan = pd.read_csv('../data/processed/jan_log.csv',parse_dates=['date'])

In [7]:
user_log_feb = pd.read_csv('../data/processed/feb_log.csv',parse_dates=['date'])

In [8]:
user_log_merge = pd.concat([user_log_jan,user_log_feb,user_log_mar],ignore_index=True)

In [10]:
user_log_jan.describe()

,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
count,1.790809e+07,1.790809e+07,1.790809e+07,1.790809e+07,1.790809e+07,1.790809e+07,1.790809e+07
mean,6.512863e+00,1.581655e+00,1.008753e+00,1.143718e+00,3.067074e+01,2.975982e+01,8.035903e+03
std,1.343888e+01,3.804299e+00,1.942348e+00,2.907919e+00,4.207879e+01,3.251800e+01,1.014903e+04
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e-03
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.000000e+00,8.000000e+00,1.963600e+03
50%,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.700000e+01,1.900000e+01,4.698940e+03
75%,7.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,3.800000e+01,3.900000e+01,1.018407e+04
max,6.437000e+03,1.077000e+03,7.470000e+02,1.125000e+03,1.524800e+04,1.887000e+03,3.743671e+06
